<a href="https://colab.research.google.com/github/monroews/playing/blob/master/Onshape_Variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install onshape_client
!pip install aguaclara


     |████████████████████████████████| 2.1MB 2.8MB/s 
     |████████████████████████████████| 112kB 23.0MB/s 
     |████████████████████████████████| 194kB 20.0MB/s 
     |████████████████████████████████| 552kB 13.9MB/s 
     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 163kB 4.9MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.1-cp36-none-any.whl size=99466 sha256=c49a51bd608cb23ef134ec63a462bf07bd6b6dac555fc109670b76527a2e7099
  Stored in directory: /root/.cache/pip/wheels/7f/ff/4f/10d76e6a261d392aefbdf6ce8d3d5519e924dc334b102ca49b
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134156 sha256=b6c9e5d99f8eaf444464136a4f6e0af7cf79e52486a8e408c808c39b2ccbffde
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint
  Found existing installation: Pint 0.12
    Uninstalling Pint-0.12:
      Successfully uninstalled Pint-0.12


In [0]:
import aguaclara as ac
from aguaclara.core.units import unit_registry as u
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


In [4]:
mylfom =  ac.LFOM(q = 10 * u.L / u.s, hl = 20 * u.cm)
print(mylfom.orifice_d.to(u.m))
print(mylfom.orifice_n_per_row)
print(ac.DRILL_BITS_D_IMPERIAL.magnitude)

0.01905 meter
[13.  3.  3.  4.  2.  3.  2.  2.  2.  2.]
[0.03125 0.0625  0.09375 0.125   0.15625 0.1875  0.21875 0.25    0.375
 0.5     0.625   0.75    0.875   1.      1.25    1.5     1.75    2.     ]


In [6]:

"""For getting measurements defined by the custom "measure" feature: https://cad.onshape.com/documents/78aa66ffe6f1daceb9cfad3d/v/025bec5f59f6380ab5627a4b/e/92549789b92e9aa35f676f4e
To fetch measurements from the API:
1. Make sure the desired geometry is defined using the feature within this document. The "use measurements variable" checkbox should be selected.
2. Call the Evaluate Featurescript API call with your client/code with the following script:
	function(context, queries) {
            return getAllVariables(context, "m");
        }
This will return all the variables defined under "m", ie, m.radius, m.my_custom_length, etc...
3. Now use the resulting map from measurements feature name to value within your code.
"""

import json

from onshape_client.oas import BTFeatureScriptEvalCall2377
from onshape_client.onshape_url import OnshapeElement
from onshape_client.utility import parse_quantity

script = r"""
    function(context, queries) {
            return getAllVariables(context);
        }
    """


def test_get_variables(client):
    element = OnshapeElement(
        "https://cad.onshape.com/documents/c3a8ce032e33ebe875b9aab4/w/de9ad5474448b34f33fef097/e/be4e3802542940dfc47aea38"
    )
    script_call = BTFeatureScriptEvalCall2377(script=script)
    response = client.part_studios_api.eval_feature_script(
        element.did,
        element.wvm,
        element.wvmid,
        element.eid,
        bt_feature_script_eval_call_2377=script_call,
        _preload_content=False,
    )
    measurements = json.loads(response.data.decode("utf-8"))["result"]["message"][
        "value"
    ]
    parsed_measurements = parse_variables_from_map(measurements)

    print("Measurements: \n" + str(parsed_measurements))


def parse_variables_from_map(unparsed):
    parsed_variables = {}
    value = None
    for to_parse in unparsed:
        if is_fs_type(to_parse, "BTFSValueMapEntry"):
            key = to_parse["message"]["key"]["message"]["value"]
            candidate_message = to_parse["message"]["value"]
            if is_fs_type(candidate_message, ["BTFSValueMap", "BTFSValueArray"]):
                value = parse_variables_from_map(candidate_message["message"]["value"])
            elif is_fs_type(candidate_message, "BTFSValueWithUnits"):
                value = parse_quantity(candidate_message["message"])
            parsed_variables[key] = value
    return parsed_variables


def is_fs_type(candidate, type_name):
    result = False
    try:
        if isinstance(type_name, str):
            result = type_name == candidate["typeName"]
        elif isinstance(type_name, list):
            result = any(
                [type_name_one == candidate["typeName"] for type_name_one in type_name]
            )
    except Exception:
        result = False
    return result


"""
Measurements: 
{'distance_from_polygon_to_square': '0.1308455707096832*meter', 'diameter': '0.0868175271040671*meter', 'polygon_side_length': '0.05706864859958895*meter'}
"""

"\nMeasurements: \n{'distance_from_polygon_to_square': '0.1308455707096832*meter', 'diameter': '0.0868175271040671*meter', 'polygon_side_length': '0.05706864859958895*meter'}\n"